Both wm1 and m7 has been filtered at high frequency for wind at 230+/- 30 aggregated at 5 mins

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
from wind_tools import *  
import pandas as pd
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib
#matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt
sns.set()
import plotly.express as px

matplotlib.rcParams['figure.figsize'] = (16,10)

In [ ]:
from scipy.spatial.transform import Rotation as R
from scipy.optimize import minimize

## Load data

In [ ]:
proc_dir = Path("2020_data/data_20200724_final_field_install/processed")

# default is 2 mins aggregated data
wm1_path = proc_dir / "eddypro_WM1_filtered_full_output_2020-08-10T141737_adv.csv"
m7_path = proc_dir / "eddypro_TRS_M507_filtered_230_30_full_output_2020-08-07T122855_exp.csv"



cache_dir = Path("data_cache")
start_date = '2020-07-24 13:00:00'
end_date = '2020-08-07 09:00:00'

wind_cols = ['u_unrot','v_unrot', 'w_unrot', 'wind_speed', 'wind_dir', 'Tau']
wind_comp_rename = {'u_unrot': 'u', 'v_unrot': 'v', 'w_unrot': 'w'}

In [ ]:
wm1, m7= map(
    lambda p: load_ep_cache(p, cache_dir= cache_dir)
    .loc[start_date:end_date]
    .loc[:,wind_cols]
    .rename(columns=wind_comp_rename),
    [wm1_path, m7_path])

In [ ]:
wm1.describe()

,u,v,w,wind_speed,wind_dir,Tau
count,743.000000,743.000000,743.000000,743.000000,743.000000,743.000000
mean,-0.517188,-0.524190,-0.037841,1.685539,139.091328,-0.064023
std,0.386897,1.720484,0.130332,0.909007,107.407128,0.126779
min,-1.501140,-3.832720,-0.545024,0.078846,3.449320,-0.631983
25%,-0.834501,-2.125830,-0.122343,0.875562,28.999150,-0.130880
50%,-0.475743,-0.273689,-0.017784,1.690980,236.165000,-0.045772
75%,-0.179550,0.865159,0.049331,2.402770,241.143500,0.003154
max,0.157657,3.246810,0.381188,4.107640,358.528000,0.484350


In [ ]:
m7.describe()

,u,v,w,wind_speed,wind_dir,Tau
count,396.000000,396.000000,396.000000,396.000000,396.000000,396.000000
mean,1.860107,0.337184,0.103020,1.997371,56.137663,-0.077071
std,0.974987,0.126241,0.183205,0.737355,59.390475,0.157672
min,-2.144150,-0.305491,-0.594317,0.116625,1.772560,-0.614078
25%,1.550875,0.258949,-0.028386,1.574158,37.105525,-0.183433
50%,2.065565,0.345121,0.088177,2.104115,39.120650,-0.089294
75%,2.455600,0.423061,0.204474,2.485350,40.574450,0.018820
max,3.558320,0.683249,0.679841,3.628410,337.866000,0.361436


In [ ]:
wm1.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-07 08:15:00', '2020-08-07 08:20:00',
               '2020-08-07 08:25:00', '2020-08-07 08:30:00',
               '2020-08-07 08:35:00', '2020-08-07 08:40:00',
               '2020-08-07 08:45:00', '2020-08-07 08:50:00',
               '2020-08-07 08:55:00', '2020-08-07 09:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3984, freq=None)

In [ ]:
m7.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-07 08:15:00', '2020-08-07 08:20:00',
               '2020-08-07 08:25:00', '2020-08-07 08:30:00',
               '2020-08-07 08:35:00', '2020-08-07 08:40:00',
               '2020-08-07 08:45:00', '2020-08-07 08:50:00',
               '2020-08-07 08:55:00', '2020-08-07 09:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3984, freq=None)

In [ ]:
m7.index

DatetimeIndex(['2020-07-24 13:05:00', '2020-07-24 13:10:00',
               '2020-07-24 13:15:00', '2020-07-24 13:20:00',
               '2020-07-24 13:25:00', '2020-07-24 13:30:00',
               '2020-07-24 13:35:00', '2020-07-24 13:40:00',
               '2020-07-24 13:45:00', '2020-07-24 13:50:00',
               ...
               '2020-08-06 23:15:00', '2020-08-06 23:20:00',
               '2020-08-06 23:25:00', '2020-08-06 23:30:00',
               '2020-08-06 23:35:00', '2020-08-06 23:40:00',
               '2020-08-06 23:45:00', '2020-08-06 23:50:00',
               '2020-08-06 23:55:00', '2020-08-07 00:00:00'],
              dtype='datetime64[ns]', name='date_time', length=3876, freq=None)

In [ ]:
plot_components([wm1, m7], ['wind_speed'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:title={'center':'wind_speed'}, xlabel='date_time'>],
      dtype=object)

In [ ]:
wm1r = rotate_wind_ang(wm1, 310)
wm1r = add_hor_wind_speed(wm1r)

# Load High freq data

In [ ]:
proc_dir = Path("2020_data/data_field_v2_from_20208010/preprocessed")
#manually filtered using custom code
wm1m_path = proc_dir / "20200810-1500_WM_174605_com1_filtered_250_30.csv"
# filtered using EddyPro
wm1ep_path = proc_dir / "../processed/filtered_2min/eddypro_raw_datasets/level_1" / "20200810-1504_raw_dataset_2020-08-13T090718_adv.txt"
wm1raw_path = proc_dir / "20200810-1500_WM_174605_com1.csv"

In [ ]:
proc_dir = Path("2020_data/data_field_v2_from_20208010/preprocessed")
#manually filtered using custom code
wm1m_path = proc_dir / "20200810-1430_WM_174605_com1_filtered_250_30.csv"
# filtered using EddyPro
wm1ep_path = proc_dir / "../processed/filtered_2min/eddypro_raw_datasets/level_1" / "20200810-1432_raw_dataset_2020-08-13T090718_adv.txt"
wm1raw_path = proc_dir / "20200810-1430_WM_174605_com1.csv"

In [ ]:
wm1m= pd.read_csv(wm1m_path)

# wm1m = wm1m[1199:2400-1].reset_index(drop=True)
wm1m = wm1m.drop(columns=['w', 't'])

In [ ]:
wm1ep = pd.read_csv(wm1ep_path, skiprows=9, delim_whitespace=True)

wm1ep = wm1ep.drop(columns=['w', 'ts'])

In [ ]:
wm1raw = pd.read_csv(wm1raw_path)
wm1raw = wm1raw.drop(columns=['w', 't'])
# wm1raw = wm1raw[1199:2400-1].reset_index(drop=True)

In [ ]:
# wm1m, wm1raw, wm1ep = map(lambda x: x.replace(-9999, np.nan), [wm1m, wm1raw, wm1ep])

## first exploration

In [ ]:
wm1raw

,u,v
0,1.09,-3.03
1,1.12,-3.46
2,1.71,-3.08
3,1.88,-3.00
4,2.05,-2.94
...,...,...
18002,0.13,-1.52
18003,0.21,-1.25
18004,-0.69,-1.34
18005,-0.79,-1.20


In [ ]:
wm1ep

,u,v
0,-9999.00,-9999.00
1,1.71,-3.08
2,1.88,-3.00
3,2.05,-2.94
4,2.00,-2.74
...,...,...
1194,-9999.00,-9999.00
1195,-9999.00,-9999.00
1196,-9999.00,-9999.00
1197,-9999.00,-9999.00


In [ ]:
wm1m

,u,v
0,-9999.0,-9999.0
1,-9999.0,-9999.0
2,-9999.0,-9999.0
3,-9999.0,-9999.0
4,-9999.0,-9999.0
...,...,...
18002,-9999.0,-9999.0
18003,-9999.0,-9999.0
18004,-9999.0,-9999.0
18005,-9999.0,-9999.0


In [ ]:
wm1m

,u,v
0,-9999.0,-9999.0
1,-9999.0,-9999.0
2,-9999.0,-9999.0
3,-9999.0,-9999.0
4,-9999.0,-9999.0
...,...,...
18002,-9999.0,-9999.0
18003,-9999.0,-9999.0
18004,-9999.0,-9999.0
18005,-9999.0,-9999.0


In [ ]:
get_wind_dir(wm1raw.u, wm1raw.v)

0        308.779050
1        304.721280
2        305.385431
3        305.487462
4        301.581292
            ...    
18002    310.689374
18003    310.044551
18004    328.535856
18005    330.875947
18006    319.763642
Length: 18007, dtype: float64

In [ ]:
wm1m[1199:2400-1]

,u,v
1199,-0.08,-1.81
1200,-0.04,-1.27
1201,0.29,-1.33
1202,0.95,-1.53
1203,0.61,-1.76
...,...,...
2394,1.56,-1.62
2395,1.36,-1.73
2396,1.50,-2.01
2397,1.16,-2.40


In [ ]:
(wm1m!=-9999).sum()

u    13
v    13
dtype: int64

In [ ]:
wm1m[wm1m!=-9999]

,u,v
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1195,NaN,NaN
1196,NaN,NaN
1197,NaN,NaN
1198,NaN,NaN


In [ ]:
(wm1m==wm1raw).u.argmax()

280

In [ ]:
wm1m.loc[280:320]

,u,v
280,-0.16,-0.92
281,-0.16,-0.94
282,0.05,-1.05
283,-9999.00,-9999.00
284,-9999.00,-9999.00
285,-9999.00,-9999.00
286,-9999.00,-9999.00
287,-9999.00,-9999.00
288,-9999.00,-9999.00
289,-9999.00,-9999.00


In [ ]:
(wm1ep==-9999).sum()

u    312
v    312
dtype: int64

In [ ]:
np.allclose(wm1m, wm1ep)

True

In [ ]:
add_wind_dir(wm1m)

,u,v,wind_dir
0,-9999.0,-9999.0,315.0
1,-9999.0,-9999.0,315.0
2,-9999.0,-9999.0,315.0
3,-9999.0,-9999.0,315.0
4,-9999.0,-9999.0,315.0
...,...,...,...
1195,-9999.0,-9999.0,315.0
1196,-9999.0,-9999.0,315.0
1197,-9999.0,-9999.0,315.0
1198,-9999.0,-9999.0,315.0


In [ ]:
add_wind_dir(wm1ep)

,u,v,wind_dir
0,-0.08,-1.81,272.530763
1,-0.04,-1.27,271.803995
2,0.29,-1.33,257.699462
3,0.95,-1.53,238.163232
4,0.61,-1.76,250.884148
...,...,...,...
1195,1.56,-1.62,226.080925
1196,1.36,-1.73,231.828150
1197,1.50,-2.01,233.267173
1198,1.16,-2.40,244.203975


In [ ]:
add_wind_dir(wm1raw)

,u,v,wind_dir
0,-0.08,-1.81,272.530763
1,-0.04,-1.27,271.803995
2,0.29,-1.33,257.699461
3,0.95,-1.53,238.163232
4,0.61,-1.76,250.884149
...,...,...,...
1195,1.56,-1.62,226.080924
1196,1.36,-1.73,231.828150
1197,1.50,-2.01,233.267173
1198,1.16,-2.40,244.203974


In [ ]:
filt = filter_by_wind_dir_single(add_wind_dir(wm1raw).wind_dir, 250, 30) 

In [ ]:
wm1raw[~filt] = -9999

In [ ]:
wm1raw

,u,v
0,-0.08,-1.81
1,-0.04,-1.27
2,0.29,-1.33
3,0.95,-1.53
4,0.61,-1.76
...,...,...
1195,1.56,-1.62
1196,1.36,-1.73
1197,1.50,-2.01
1198,1.16,-2.40


In [ ]:
wm1ep

,u,v
0,-0.08,-1.81
1,-0.04,-1.27
2,0.29,-1.33
3,0.95,-1.53
4,0.61,-1.76
...,...,...
1195,1.56,-1.62
1196,1.36,-1.73
1197,1.50,-2.01
1198,1.16,-2.40


In [ ]:
np.allclose(wm1raw, wm1ep)

True

In [ ]:
mod(get_wind_dir(wm1ep.u, wm1ep.v) + 310)

0       265.000000
1       265.000000
2       253.151733
3       251.204748
4       249.872204
           ...    
1195    265.000000
1196    265.000000
1197    265.000000
1198    265.000000
1199    265.000000
Length: 1200, dtype: float64

In [ ]:
(wm1m != wm1raw).any()

u    False
v    False
dtype: bool

In [ ]:
wm1m.u.to_numpy()

array([-2.92, -2.78, -2.26, ..., -0.94, -0.97, -0.92])

In [ ]:
len(wm1raw)

1200

In [ ]:
mod( get_wind_dir(wm1m.u, wm1m.v) + 310)

0       265.000000
1       267.695551
2       253.151734
3       251.204747
4       249.872205
           ...    
1195    277.578935
1196    283.434949
1197    296.827447
1198    290.683647
1199    272.341226
Length: 1200, dtype: float64

In [ ]:
wm1m.isnull().any()

u    False
v    False
dtype: bool

In [ ]:
mod( get_wind_dir(wm1raw.u, wm1raw.v) + 310)

0       265.000000
1       267.695551
2       253.151734
3       251.204747
4       249.872205
           ...    
1195    277.578935
1196    283.434949
1197    296.827447
1198    290.683647
1199    272.341226
Length: 1200, dtype: float64

In [ ]:
filt = filter_by_wind_dir_single(mod( get_wind_dir(wm1m.u, wm1m.v) + 310), 230, 30) 

In [ ]:
wm1m[~filt] = -9999

In [ ]:
np.allclose(wm1ep, wm1m)

True

In [ ]:
wm1m

,u,v
0,NaN,NaN
1,NaN,NaN
2,-2.26,-3.46
3,-1.69,-2.79
4,-1.39,-2.42
...,...,...
1195,NaN,NaN
1196,NaN,NaN
1197,NaN,NaN
1198,NaN,NaN
